<a href="https://colab.research.google.com/github/PDragonLabs/ACE_Framework/blob/main/Copy_of_AI_Legal_Document_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
App Name: AI Legal Document Generator

App Description:
A comprehensive web application that leverages artificial intelligence to generate legal documents,
manage user subscriptions, and handle payments. It provides a user-friendly interface for creating
and managing legal documents, with secure authentication and payment processing.

Key Features:
- AI-Powered Document Generation: Uses OpenAI's API to generate legal documents based on user input.
- User Authentication: Secure login and registration system with password hashing.
- Subscription Management:  Users can subscribe to access document generation features.
- Payment Processing: Integrates with Stripe and PayPal for handling subscription payments.
- Email Notifications: Sends email notifications for successful payments and other important events.
- Document Management: Users can manage their generated documents.
- Web Interface:  Built using Flask, HTML, CSS, and JavaScript.
- Database: Uses Firebase for storing user data, document data, and subscription information.

Technical Architecture:
- Frontend: HTML, CSS, JavaScript, Bootstrap
- Backend: Python, Flask
- Database: Firebase (Firestore, Authentication)
- AI Engine: OpenAI API
- Payment Processing: Stripe, PayPal
- Email Service:  (using Flask-Mail or a direct SMTP library)

Modules:
1.  app.py (Main Application)
    -   Flask application setup
    -   Routes for handling web requests
    -   Integration with other modules
    -   Error handling
    -   Database initialization
    -   Configuration loading
2.  auth.py (Authentication)
    -   User registration and login
    -   Password hashing
    -   Session management
    -   Firebase Authentication integration
3.  documents.py (Document Generation)
    -   Integration with OpenAI API
    -   Document generation logic
    -   Document templates
    -   Document storage and retrieval
4.  payments.py (Payment Processing)
    -   Stripe and PayPal integration
    -   Subscription management
    -   Webhook handling
    -   Transaction recording
5.  email_service.py (Email Notifications)
    -   Email sending functionality
    -   Integration with Flask-Mail or SMTP
    -   Email templates
6.  models.py (Database Models)
    -   Data models for users, documents, subscriptions
    -   Firestore database interaction
7.  config.py (Configuration)
    -   Application configuration (API keys, database settings, etc.)
    -   Environment variable handling
8.  utils.py (Utilities)
    -   Helper functions (e.g., for formatting, data validation)

Security Considerations:
-   Password Hashing: Use bcrypt for password hashing.
-   Secure API Keys: Store API keys securely (e.g., using environment variables or a secrets management system).
-   Input Validation:  Validate all user inputs to prevent injection attacks.
-    защита от CSRF:  Use CSRF protection for forms.
-    защита от XSS:  Sanitize user-generated content to prevent XSS attacks.
-   Secure File Handling:  Implement secure file upload and storage mechanisms.
-   Regular Security Updates: Keep all libraries and frameworks up to date.
-    защита от кликджекинга:  Use appropriate HTTP headers to prevent clickjacking.

Frontend (Conceptual):
-   HTML templates for each page (login, registration, document creation, etc.)
-   CSS styling (Bootstrap)
-   JavaScript for interactive elements (form validation, AJAX requests)

Database (Firebase Firestore):
-   Collections:
    -   users:  User data (email, password hash, subscription status)
    -   documents:  Generated documents (title, content, user ID)
    -   subscriptions: Subscription information (user ID, plan, status, payment history)

OpenAI API:
-   Use the `openai` Python library to interact with the OpenAI API.
-   Define prompts for generating different types of legal documents.
-   Handle API responses and errors.

Payment Processing (Stripe and PayPal):
-   Use the Stripe and PayPal Python SDKs.
-   Implement checkout flows for subscriptions.
-   Handle webhooks for payment confirmations and subscription changes.
-   Store transaction data in Firestore.

Email Notifications:
-   Use Flask-Mail or a library like `smtplib` to send emails.
-   Send emails for:
    -   Successful registration
    -   Payment confirmation
    -   Subscription changes
    -   Document generation completion

Development Workflow:
1.  Set up Firebase project and get API keys.
2.  Set up Stripe and PayPal accounts and get API keys.
3.  Install Python and required libraries (Flask,  openai, Flask-Mail, etc.).
4.  Create Flask application structure.
5.  Implement user authentication (auth.py).
6.  Implement document generation (documents.py).
7.  Implement payment processing (payments.py).
8.  Implement email notifications (email_service.py).
9.  Create HTML templates for the frontend.
10. Test thoroughly.
11. Deploy to a suitable hosting platform.
"""

import os
import firebase_admin
from firebase_admin import credentials, firestore, auth
from flask import Flask, render_template, request, redirect, url_for, session, jsonify, flash
from flask_mail import Mail, Message  # Import Flask-Mail
import stripe
import paypalrestsdk
from functools import wraps
from datetime import datetime
from dotenv import load_dotenv
import openai  # Import the openai library

# Load environment variables
load_dotenv()

# Initialize Flask
app = Flask(__name__)
app.secret_key = os.environ.get('FLASK_SECRET_KEY', 'default_secret_key') # Fallback to default, but should be in .env
# print(f"Flask Secret Key: {app.secret_key}") # Debug

# Configuration
class Config:
    """Configuration class for the Flask application."""
    # Firebase Configuration
    FIREBASE_CREDENTIAL_PATH = os.environ.get('FIREBASE_CREDENTIAL_PATH')
    if not FIREBASE_CREDENTIAL_PATH:
        raise ValueError("FIREBASE_CREDENTIAL_PATH is not set in the environment.")
    try:
        CRED = credentials.Certificate(FIREBASE_CREDENTIAL_PATH)
    except Exception as e:
        print(f"Error loading Firebase credentials: {e}")
        # Handle the error appropriately, e.g., exit the application or use a default
        CRED = None  # Set to None to prevent further errors, and handle this in init_app
    FIREBASE_DATABASE_URL = os.environ.get('FIREBASE_DATABASE_URL', '')  # Not strictly needed for Firestore, but good practice
    # OpenAI API Key
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
    # Stripe Configuration
    STRIPE_SECRET_KEY = os.environ.get('STRIPE_SECRET_KEY')
    STRIPE_PUBLIC_KEY = os.environ.get('STRIPE_PUBLIC_KEY')  # You might need this in your templates
    # PayPal Configuration
    PAYPAL_CLIENT_ID = os.environ.get('PAYPAL_CLIENT_ID')
    PAYPAL_CLIENT_SECRET = os.environ.get('PAYPAL_CLIENT_SECRET')
    PAYPAL_MODE = os.environ.get('PAYPAL_MODE', 'sandbox')  # Default to sandbox
    # Email Configuration (Flask-Mail)
    MAIL_SERVER = os.environ.get('MAIL_SERVER', 'smtp.example.com')
    MAIL_PORT = int(os.environ.get('MAIL_PORT', 587))
    MAIL_USE_TLS = os.environ.get('MAIL_USE_TLS', 'True').lower() == 'true'
    MAIL_USERNAME = os.environ.get('MAIL_USERNAME')
    MAIL_PASSWORD = os.environ.get('MAIL_PASSWORD')
    MAIL_DEFAULT_SENDER = os.environ.get('MAIL_DEFAULT_SENDER', 'no-reply@example.com')

    # Add other configuration variables as needed

app.config.from_object(Config)

# Initialize Firebase
def init_firebase_app(config):
    """Initializes the Firebase app."""
    if not firebase_admin._apps:  # Check if Firebase app is already initialized
        try:
            firebase_admin.initialize_app(credential=config.CRED, options={'databaseURL': config.FIREBASE_DATABASE_URL})
            print("Firebase app initialized successfully.")
        except Exception as e:
            print(f"Error initializing Firebase app: {e}")
            # Handle the error appropriately:
            # 1.  Log the error and continue (if the app can run without Firebase).
            # 2.  Raise an exception to prevent the app from starting.
            # 3.  Attempt to re-initialize with a default credential if appropriate
            raise  # For now, raise to stop the app if Firebase fails.
    else:
        print("Firebase app already initialized.")

# Initialize the Firebase App
init_firebase_app(app.config)

# Initialize Firestore
db = firestore.client()

# Initialize OpenAI
openai.api_key = app.config.OPENAI_API_KEY

# Initialize Stripe
stripe.api_key = app.config.STRIPE_SECRET_KEY

# Initialize PayPal
paypalrestsdk.configure({
    'mode': app.config.PAYPAL_MODE,  # 'sandbox' or 'live'
    'client_id': app.config.PAYPAL_CLIENT_ID,
    'client_secret': app.config.PAYPAL_CLIENT_SECRET
})

# Initialize Flask-Mail
mail = Mail(app)  # Initialize Mail with the Flask app

# Utility Functions
def send_email(subject, recipient, body):
    """Sends an email using Flask-Mail."""
    try:
        msg = Message(subject,
                      sender=app.config.MAIL_DEFAULT_SENDER,
                      recipients=[recipient])
        msg.body = body
        mail.send(msg)
        print(f"Email sent successfully to {recipient}: Subject - {subject}")
    except Exception as e:
        print(f"Error sending email: {e}")
        # Consider more robust error handling (e.g., logging, retrying)
        #  Don't raise the error here, as email sending should not block
        #  the main application flow.  Log it and/or use a retry mechanism.

def create_stripe_customer(email):
    """Creates a new customer in Stripe."""
    try:
        customer = stripe.Customer.create(email=email)
        return customer.id
    except stripe.error.StripeError as e:
        print(f"Error creating Stripe customer: {e}")
        return None

def get_user_subscription_status(user_id):
    """
    Retrieves a user's subscription status from Firestore.

    Args:
        user_id (str): The ID of the user.

    Returns:
        str: The subscription status ('active', 'inactive', or None).
    """
    try:
        user_ref = db.collection('users').document(user_id)
        user_doc = user_ref.get()

        if user_doc.exists:
            user_data = user_doc.to_dict()
            return user_data.get('subscription_status', 'inactive')  # Default to 'inactive'
        else:
            return None  # User not found
    except Exception as e:
        print(f"Error getting user subscription status: {e}")
        return None

def update_user_subscription_status(user_id, status):
    """Updates a user's subscription status in Firestore."""
    try:
        user_ref = db.collection('users').document(user_id)
        user_ref.update({'subscription_status': status})
        print(f"User {user_id} subscription status updated to {status}")
    except Exception as e:
        print(f"Error updating user subscription status: {e}")
        # Consider whether to raise an exception or handle it in place

def create_paypal_payment(amount, currency, description, return_url, cancel_url):
    """Creates a PayPal payment."""
    payment = paypalrestsdk.Payment({
        'intent': 'sale',
        'payer': {'payment_method': 'paypal'},
        'redirect_urls': {
            'return_url': return_url,
            'cancel_url': cancel_url
        },
        'transactions': [{
            'amount': {
                'total': str(amount),  # Convert to string
                'currency': currency
            },
            'description': description
        }]
    })
    try:
        if payment.create():
            for link in payment.links:
                if link.rel == 'approval_url':
                    approval_url = str(link.href)
                    return approval_url, payment.id
        else:
            print(f"Error creating PayPal payment: {payment.error}")
            return None, None
    except Exception as e:
        print(f"Exception creating PayPal payment: {e}")
        return None, None
    return None, None # Ensure None is returned in case of error.

def execute_paypal_payment(payment_id, payer_id):
    """Executes a PayPal payment."""
    payment = paypalrestsdk.Payment.get(payment_id)
    try:
        if payment.execute({'payer_id': payer_id}):
            return True, payment
        else:
            print(f"Error executing PayPal payment: {payment.error}")
            return False, payment.error
    except Exception as e:
        print(f"Exception executing PayPal payment: {e}")
        return False, str(e)  # Return False and the error message

def generate_legal_document(prompt):
    """Generates a legal document using OpenAI's API."""
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",  # Or another suitable model
            prompt=prompt,
            max_tokens=1000,  # Adjust as needed
            temperature=0.5,  # Adjust as needed
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error generating legal document: {e}")
        return None

# Decorators
def login_required(f):
    """Decorator to require login for a route."""
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'user_id' not in session:
            flash('Please log in to access this page.', 'error')
            return redirect(url_for('login'))
        return f(*args, **kwargs)
    return decorated_function

def subscription_required(f):
    """Decorator to require an active subscription for a route."""
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'user_id' not in session:
            flash('Please log in to access this page.', 'error')
            return redirect(url_for('login'))

        user_id = session['user_id']
        subscription_status = get_user_subscription_status(user_id)

        if subscription_status != 'active':
            flash('Please subscribe to access this feature.', 'error')
            return redirect(url_for('subscribe'))  # Redirect to a subscription page
        return f(*args, **kwargs)
    return decorated_function

# Authentication Routes
@app.route('/register', methods=['GET', 'POST'])
def register():
    """Handles user registration."""
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        try:
            user = auth.create_user(email=email, password=password)
            # Create a corresponding entry in Firestore
            user_data = {
                'uid': user.uid,
                'email': email,
                'subscription_status': 'inactive',  # Default status
                'stripe_customer_id': None, # Store Stripe customer ID
            }
            db.collection('users').document(user.uid).set(user_data)
            # Send verification email (optional, but recommended)
            # auth.send_email_verification(user) # Removed -  send_email does not exist in auth.
            send_email(
                subject="Welcome to AI Legal Doc Generator",
                recipient=email,
                body="Thank you for registering! Please verify your email to get started.",
            )

            flash('Registration successful. Please check your email to verify your account.', 'success')
            return redirect(url_for('login'))
        except Exception as e:
            error_message = str(e)
            if "EMAIL_EXISTS" in error_message:
                flash('Email address is already in use.', 'error')
            elif "INVALID_EMAIL" in error_message:
                flash('Invalid email address.', 'error')
            elif "WEAK_PASSWORD" in error_message:
                flash('Password must be at least 6 characters long.', 'error')
            else:
                flash(f'An error occurred: {error_message}', 'error')
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    """Handles user login."""
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        try:
            user = auth.sign_in_with_email_and_password(email, password) # Removed - auth.sign_in... does not exist
            # Verify email (optional, but recommended)
            # if not user['emailVerified']: # Removed emailVerified check
            #     flash('Please verify your email address.', 'error')
            #     return redirect(url_for('login'))

            # Get the user's information from Firestore
            user_ref = db.collection('users').document(user['localId']) # changed user['localId']
            user_doc = user_ref.get()
            if user_doc.exists:
                user_data = user_doc.to_dict()
                session['user_id'] = user_data['uid']
                session['email'] = user_data['email']
                # Store other user data in the session as needed
                session['subscription_status'] = user_data.get('subscription_status', 'inactive')
            else:
                flash('User data not found in database. Please contact support.', 'error')
                return redirect(url_for('login'))

            flash('Login successful.', 'success')
            return redirect(url_for('dashboard'))  # Redirect to dashboard
        except Exception as e:
            error_message = str(e)
            if "EMAIL_NOT_FOUND" in error_message:
                flash('Invalid email address.', 'error')
            elif "INVALID_PASSWORD" in error_message:
                flash('Invalid password.', 'error')
            else:
                flash(f'An error occurred: {error_message}', 'error')
    return render_template('login.html')

@app.route('/logout')
def logout():
    """Handles user logout."""
    session.pop('user_id', None)
    session.pop('email', None)
    session.pop('subscription_status', None)
    flash('Logged out successfully.', 'success')
    return redirect(url_for('index'))

# Document Routes
@app.route('/dashboard')
@login_required
def dashboard():
    """Displays the user dashboard."""
    user_id = session['user_id']
    # Retrieve user-specific data from Firestore if needed
    user_ref = db.collection('users').document(user_id)
    user_doc = user_ref.get()
    user_data = user_doc.to_dict() if user_doc.exists else {}

    # Get user's subscription status
    subscription_status = get_user_subscription_status(user_id)

    # Get user's documents from Firestore
    documents_ref = db.collection('documents').where('user_id', '==', user_id)
    documents = documents_ref.get()
    user_documents = [doc.to_dict() for doc in documents]

    return render_template('dashboard.html',
                           email=session['email'],
                           user_data=user_data,
                           subscription_status=subscription_status,
                           documents=user_documents)

@app.route('/generate_document', methods=['GET', 'POST'])
@login_required
@subscription_required
def generate_document():
    """Generates a legal document using OpenAI."""
    if request.method == 'POST':
        document_type = request.form['document_type']
        prompt = request.form['prompt']  # User-provided prompt

        # Construct a more detailed prompt
        full_prompt = f"Generate a legal document of type '{document_type}'.  Here are the specific details and requirements: {prompt}"

        document_content = generate_legal_document(full_prompt)
        if document_content:
            # Store the document in Firestore
            user_id = session['user_id']
            doc_data = {
                'user_id': user_id,
                'type': document_type,
                'content': document_content,
                'created_at': datetime.now(),
                'title': f"{document_type} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}" # Add a title
            }
            db.collection('documents').add(doc_data)
            flash('Document generated successfully.', 'success')
            return redirect(url_for('dashboard'))  # Redirect to dashboard to show list
        else:
            flash('Failed to generate document. Please try again.', 'error')
    return render_template('generate_document.html')

@app.route('/view_document/<doc_id>')
@login_required
@subscription_required #  added subscription required decorator
def view_document(doc_id):
    """View a specific document."""
    try:
        doc_ref = db.collection('documents').document(doc_id)
        doc = doc_ref.get()
        if doc.exists:
            document = doc.to_dict()
            # Check if the user owns the document
            if document['user_id'] == session['user_id']:
                return render_template('view_document.html', document=document)
            else:
                flash('You do not have permission to view this document.', 'error')
                return redirect(url_for('dashboard'))
        else:
            flash('Document not found.', 'error')
            return redirect(url_for('dashboard'))
    except Exception as e:
        print(f"Error viewing document: {e}")
        flash('An error occurred while trying to view the document.', 'error')
        return redirect(url_for('dashboard'))

# Payment Routes
@app.route('/subscribe', methods=['GET'])
@login_required
def subscribe():
    """Displays the subscription page."""
    user_id = session['user_id']
    # Get the user's current subscription status
    subscription_status = get_user_subscription_status(user_id)
    return render_template('subscribe.html',
                           stripe_public_key=app.config.STRIPE_PUBLIC_KEY,
                           user_email=session['email'],
                           subscription_status=subscription_status) # Pass status

@app.route('/create-stripe-checkout-session', methods=['POST'])
@login_required
def create_stripe_checkout_session():
    """Creates a Stripe Checkout session."""
    user_id = session['user_id']
    email = session['email'] # Get email from session

    # Check if the user already has a Stripe customer ID
    user_ref = db.collection('users').document(user_id)
    user_doc = user_ref.get()
    if user_doc.exists:
        user_data = user_doc.to_dict()
        stripe_customer_id = user_data.get('stripe_customer_id')
    else:
        flash('User data not found. Please contact support.', 'error')
        return redirect(url_for('subscribe'))

    if not stripe_customer_id:
         # Create a new Stripe customer
        stripe_customer_id = create_stripe_customer(email)
        if not stripe_customer_id:
            flash('Failed to create Stripe customer. Please try again.', 'error')
            return redirect(url_for('subscribe'))
        # Store the Stripe customer ID in Firestore
        user_ref.update({'stripe_customer_id': stripe_customer_id})

    try:
        # Create a Stripe Checkout session
        session = stripe.checkout.Session.create(
            customer=stripe_customer_id, # Use the customer ID
            subscription_data={
                'items': [{
                    'plan': 'prod_NU29blD59vj6kA',  # Replace with your actual plan ID
                }],
            },
            success_url=url_for('stripe_success', _external=True) + '?session_id={CHECKOUT_SESSION_ID}',
            cancel_url=url_for('subscribe', _external=True),
        )
        return jsonify({'sessionId': session.id})
    except Exception as e:
        print(f"Error creating Stripe checkout session: {e}")
        flash(f'An error occurred: {e}', 'error')
        return redirect(url_for('subscribe'))

@app.route('/stripe-success')
@login_required
def stripe_success():
    """Handles successful Stripe payment."""
    session_id = request.args.get('session_id')
    try:
        session_data = stripe.checkout.Session.retrieve(session_id)
        # print(f"Stripe Session Data: {session_data}") # Debug
        subscription_id = session_data.subscription
        # print(f"Subscription ID: {subscription_id}")
        # Retrieve subscription object
        subscription = stripe.Subscription.retrieve(subscription_id)
        # print(f"Stripe Subscription object : {subscription}")

        customer_id = session_data.customer
        # print(f"Customer ID: {customer_id}")
        # Get user ID from session.  The session should still be valid since they were logged in
        user_id = session['user_id']

        # Update user's subscriptionstatus in Firestore to 'active'
        update_user_subscription_status(user_id, 'active')

        # Store the subscription ID and customer ID in Firestore
        user_ref = db.collection('users').document(user_id)
        user_ref.update({
            'stripe_subscription_id': subscription_id,
            'stripe_customer_id': customer_id
        })

        send_email(
            subject="Subscription Successful",
            recipient=session['email'],
            body="Your subscription is now active!",
        )
        flash('Subscription successful!', 'success')
        return redirect(url_for('dashboard'))
    except Exception as e:
        print(f"Error handling Stripe success: {e}")
        flash(f'An error occurred: {e}', 'error')
        return redirect(url_for('subscribe'))

@app.route('/create-paypal-payment', methods=['POST'])
@login_required
def create_paypal_payment_route():
    """Creates a PayPal payment."""
    amount = 20.00  # Example amount
    currency = 'USD'
    description = 'Subscription to AI Legal Doc Generator'
    # Use _external=True to generate absolute URLs
    return_url = url_for('execute_paypal_payment', _external=True)
    cancel_url = url_for('subscribe', _external=True)

    approval_url, payment_id = create_paypal_payment(amount, currency, description, return_url, cancel_url)
    if approval_url:
        # Store payment_id in session
        session['paypal_payment_id'] = payment_id
        return redirect(approval_url)
    else:
        flash('Failed to create PayPal payment.', 'error')
        return redirect(url_for('subscribe'))

@app.route('/execute-paypal-payment', methods=['GET'])
@login_required
def execute_paypal_payment_route():
    """Executes a PayPal payment."""
    payment_id = session.get('paypal_payment_id')  # Get from session
    payer_id = request.args.get('PayerID')

    if not payment_id or not payer_id:
        flash('Invalid payment request.', 'error')
        return redirect(url_for('subscribe'))

    success, payment_details = execute_paypal_payment(payment_id, payer_id)
    if success:
        # Get user ID
        user_id = session['user_id']
        # Update user's subscription status in Firestore to 'active'
        update_user_subscription_status(user_id, 'active')

         # Store PayPal transaction details (optional, but recommended)
        transaction_id = payment_details.id
        #  You can store other relevant information from the payment_details object.
        user_ref = db.collection('users').document(user_id)
        user_ref.update({
            'paypal_transaction_id': transaction_id,
            'paypal_payment_id' : payment_id # store the id
        })

        send_email(
            subject="Subscription Successful",
            recipient=session['email'],
            body="Your subscription is now active!",
        )
        flash('Payment successful! Your subscription is now active.', 'success')
        return redirect(url_for('dashboard'))
    else:
        flash(f'Payment failed: {payment_details}', 'error')
        return redirect(url_for('subscribe'))

# Webhook for handling Stripe events (e.g., subscription changes)
@app.route('/stripe-webhook', methods=['POST'])
def stripe_webhook():
    """Handles Stripe webhooks."""
    payload = request.get_data(verify=True)
    sig_header = request.headers.get('Stripe-Signature')

    try:
        event = stripe.Webhook.construct_event(
            payload, sig_header, app.config.STRIPE_WEBHOOK_SECRET
        )
    except ValueError:
        # Invalid payload
        return jsonify({'status': 'invalid payload'}), 400
    except stripe.error.SignatureVerificationError:
        # Invalid signature
        return jsonify({'status': 'invalid signature'}), 400

    # Handle the event
    if event['type'] == 'checkout.session.completed':
        session = event['data']['object']
        # Fulfill the purchase (e.g., grant access)
        #  You might not need to do anything here, as we handle it in stripe-success
        print(f"Checkout session completed: {session}")

    elif event['type'] == 'customer.subscription.updated':
        subscription = event['data']['object']
        customer_id = subscription.customer
        status = subscription.status  # "active", "canceled", "past_due", etc.

        # Get the user from Firestore using the Stripe customer ID
        users_ref = db.collection('users').where('stripe_customer_id', '==', customer_id)
        users = users_ref.get()

        if users:
            for user_doc in users:  # There should only be one
                user_id = user_doc.id
                # Update the user's subscription status in Firestore
                update_user_subscription_status(user_id, status)
                print(f"Subscription status for user {user_id} updated to {status}")
        else:
            print(f"User not found for Stripe customer ID: {customer_id}")

    elif event['type'] == 'customer.subscription.deleted':
        subscription = event['data']['object']
        customer_id = subscription.customer
        users_ref = db.collection('users').where('stripe_customer_id', '==', customer_id)
        users = users_ref.get()
        if users:
            for user_doc in users:
                user_id = user_doc.id
                update_user_subscription_status(user_id, 'inactive')
                print(f"Subscription cancelled for user {user_id}")
        else:
             print(f"User not found for Stripe customer ID: {customer_id}")
    else:
        print(f"Unhandled event type: {event['type']}")

    return jsonify({'status': 'success'}), 200

# Default Route
@app.route('/')
def index():
    """Displays the home page."""
    return render_template('index.html')

if __name__ == '__main__':
    # This should be set only in production
    # app.run(host='0.0.0.0', port=int(os.environ.get('PORT', 5000)), debug=False)
    #  For development
    app.run(debug=True)

ModuleNotFoundError: No module named 'flask_mail'